In [2]:
!nvidia-smi

Mon Mar  7 21:29:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 匯入函式庫

In [3]:
!pip install openpyxl==3.0.9

In [4]:

import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from statsmodels.stats.diagnostic import acorr_ljungbox


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# CPU/GPU、自定義資料集、模型、訓練函數

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [7]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = torch.from_numpy(X)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [8]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [9]:
class Self_Attention(nn.Module):
    def __init__(self, num_layers, hidden_size, length_input_sequence):
        super().__init__()
        self.num_layers = num_layers
        
        self.Input_First_HiddenLayer = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1)
        self.Query_Key_Value_1 = nn.ModuleList([nn.Linear(1, hidden_size), nn.Linear(1, hidden_size), nn.Linear(1, hidden_size)])

        self.Second_And_Following_HiddenLayer = nn.ModuleList([nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1) for i in range(num_layers - 1)])
        self.List_of_Query_Key_Value = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_size, hidden_size) for q_k_v in range(3)]) for i in range(num_layers - 1)])

        self.OutputLayer = nn.Linear(hidden_size, 1)

        # generate mask
        mask = (torch.triu(torch.ones(length_input_sequence, length_input_sequence)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        self.register_buffer('mask', mask)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        Query1 = self.Query_Key_Value_1[0](input)
        Key1 = self.Query_Key_Value_1[1](input)
        Value1 = self.Query_Key_Value_1[2](input)
        hidden, _ = self.Input_First_HiddenLayer(Query1, Key1, Value1, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        if self.num_layers > 1:
            for i, Second_And_Following_HiddenLayer in enumerate(self.Second_And_Following_HiddenLayer):
                Query = self.List_of_Query_Key_Value[i][0](hidden)
                Key = self.List_of_Query_Key_Value[i][1](hidden)
                Value = self.List_of_Query_Key_Value[i][2](hidden)
                hidden, _ = Second_And_Following_HiddenLayer(Query, Key, Value, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [10]:
def train_under_config(model_type,
                       forex_data,
                       length_input_sequence,
                       num_epochs,
                       num_hidden_layers,
                       num_hidden_sizes,
                       batch_sizes,
                       device):
    '''
    model_type: LSTM or Self_Attention,
    forex_data,
    length_input_sequence,
    num_epochs,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    # dataset
    training_data = forex_data.loc['1981-01-01':'2016-12-31'] # training_data = training set + validation set
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataloader
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # model
    if model_type == Self_Attention:
        model = model_type(num_hidden_layers, num_hidden_sizes, length_input_sequence).double()
    else:
        model = model_type(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    # training
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
    
    return model

In [11]:
# residue pred
def ypred(data, length_input_sequence, model, device):
    '''
    data,
    length_input_sequence,
    model,
    device
    '''
    # dataset
    testing_start_index = len(data.loc['1981-01-01':'2016-12-31']) - length_input_sequence
    testing_end_index = len(data.loc['1981-01-01':'2020-12-31'])
    testing_data = data[testing_start_index:testing_end_index]
    testing_dataset = TimeSeriesDataset(testing_data, length_input_sequence)
    # dataloader
    data_len = len(testing_dataset)
    testing_dataloader = DataLoader(testing_dataset, batch_size=data_len, shuffle=False)
    # evaluating
    model.to(device)
    model.eval()
    for X, y in  testing_dataloader:
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            ypred = model(X)
            ypred = np.array(ypred.cpu()).flatten()
    return ypred

In [12]:
def prediction_on_testset(model_type, linear_data, residue_data, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs):
    model = train_under_config(model_type,
                               residue_data,
                               length_input_sequence,
                               num_epochs,
                               num_hidden_layers,
                               num_hidden_sizes,
                               batch_sizes,
                               device)
    model_type_linear_pred = np.array(linear_data.loc['2017-01-01':'2020-12-31'])
    model_type_residue_pred = ypred(residue_data, length_input_sequence, model, device)
    model_type_pred = model_type_linear_pred + model_type_residue_pred
    return model_type_pred

In [13]:
# random walk prediction
def rw_pred(testing_data):
    return np.zeros(len(testing_data))

In [14]:
def root_mean_squared_error(ypred, ytrue):
    squared_error = (ypred - ytrue) ** 2
    mean_squared_error = sum(squared_error) / len(squared_error)
    return np.sqrt(mean_squared_error)

In [15]:
def mean_absolute_error(ypred, ytrue):
    absolute_error = abs(ypred - ytrue)
    return sum(absolute_error) / len(absolute_error)

In [16]:
def direction_accuracy(ypred, ytrue):
    ypred_elementwise_dot_ytrue = ypred * ytrue
    same_direction_rate = sum(ypred_elementwise_dot_ytrue > 0) / len(ypred_elementwise_dot_ytrue)
    return same_direction_rate

# 匯入訓練資料（訓練加驗證集）訓練，並產生模型預測值

## Recursive window

In [17]:
recursive_linear = pd.read_csv('./recursive_linear_prediction.csv', index_col=0)
recursive_linear = recursive_linear.astype('float64')
recursive_linear.index = pd.to_datetime(recursive_linear.index)

cad_recursive_linear = recursive_linear.iloc[:,0]
aud_recursive_linear = recursive_linear.iloc[:,1]
gbp_recursive_linear = recursive_linear.iloc[:,2]

recursive_linear

,cad_recursive_linear_prediction,aud_recursive_linear_prediction,gbp_recursive_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000012,-0.000047,0.000109
2020-12-28,-0.000012,-0.000047,-0.000048
2020-12-29,-0.000012,-0.000048,-0.000323
2020-12-30,-0.000012,-0.000047,0.000074


In [18]:
recursive_residue = pd.read_csv('./recursive_linear_residue.csv', index_col=0)
recursive_residue = recursive_residue.astype('float64')
recursive_residue.index = pd.to_datetime(recursive_residue.index)

cad_recursive_residue = recursive_residue.iloc[:,0]
aud_recursive_residue = recursive_residue.iloc[:,1]
gbp_recursive_residue = recursive_residue.iloc[:,2]

recursive_residue

,cad_recursive_linear_residue,aud_recursive_linear_residue,gbp_recursive_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000117,0.001100,0.000038
2020-12-28,0.001426,-0.003377,-0.007949
2020-12-29,0.002194,0.003604,0.004107
2020-12-30,0.005254,0.010510,0.008920


### LSTM

In [19]:
model_type = LSTM

#### cad

In [20]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [21]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [22]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [23]:
cad_recursive_LSTM_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [24]:
aud_recursive_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000047
2020-12-28   -0.000047
2020-12-29   -0.000048
2020-12-30   -0.000047
2020-12-31   -0.000046
Name: aud_recursive_linear_prediction, Length: 10435, dtype: float64

In [25]:
aud_recursive_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001100
2020-12-28   -0.003377
2020-12-29    0.003604
2020-12-30    0.010510
2020-12-31    0.001087
Name: aud_recursive_linear_residue, Length: 10435, dtype: float64

In [26]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 3
num_hidden_sizes = 1600
batch_sizes = 128
num_epochs = 15

In [27]:
aud_recursive_LSTM_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [28]:
gbp_recursive_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000109
2020-12-28   -0.000048
2020-12-29   -0.000323
2020-12-30    0.000074
2020-12-31    0.000251
Name: gbp_recursive_linear_prediction, Length: 10435, dtype: float64

In [29]:
gbp_recursive_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25    0.000038
2020-12-28   -0.007949
2020-12-29    0.004107
2020-12-30    0.008920
2020-12-31    0.003047
Name: gbp_recursive_linear_residue, Length: 10435, dtype: float64

In [30]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 25
batch_sizes = 256
num_epochs = 5

In [31]:
gbp_recursive_LSTM_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [32]:
model_type = Self_Attention

#### cad

In [33]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [34]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [35]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 15

In [36]:
cad_recursive_Self_Attention_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [37]:
aud_recursive_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000047
2020-12-28   -0.000047
2020-12-29   -0.000048
2020-12-30   -0.000047
2020-12-31   -0.000046
Name: aud_recursive_linear_prediction, Length: 10435, dtype: float64

In [38]:
aud_recursive_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001100
2020-12-28   -0.003377
2020-12-29    0.003604
2020-12-30    0.010510
2020-12-31    0.001087
Name: aud_recursive_linear_residue, Length: 10435, dtype: float64

In [39]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 1
num_hidden_sizes = 800
batch_sizes = 16
num_epochs = 30

In [40]:
aud_recursive_Self_Attention_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [41]:
gbp_recursive_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000109
2020-12-28   -0.000048
2020-12-29   -0.000323
2020-12-30    0.000074
2020-12-31    0.000251
Name: gbp_recursive_linear_prediction, Length: 10435, dtype: float64

In [42]:
gbp_recursive_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25    0.000038
2020-12-28   -0.007949
2020-12-29    0.004107
2020-12-30    0.008920
2020-12-31    0.003047
Name: gbp_recursive_linear_residue, Length: 10435, dtype: float64

In [43]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 20

In [44]:
gbp_recursive_Self_Attention_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

## Rolling window

In [45]:
rolling_linear = pd.read_csv('./rolling_linear_prediction.csv', index_col=0)
rolling_linear = rolling_linear.astype('float64')
rolling_linear.index = pd.to_datetime(rolling_linear.index)

cad_rolling_linear = rolling_linear.iloc[:,0]
aud_rolling_linear = rolling_linear.iloc[:,1]
gbp_rolling_linear = rolling_linear.iloc[:,2]

rolling_linear

,cad_rolling_linear_prediction,aud_rolling_linear_prediction,gbp_rolling_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000002,-0.000015,0.000171
2020-12-28,-0.000002,-0.000014,0.000020
2020-12-29,-0.000002,-0.000014,-0.000245
2020-12-30,-0.000002,-0.000014,0.000138


In [46]:
rolling_residue = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
rolling_residue = rolling_residue.astype('float64')
rolling_residue.index = pd.to_datetime(rolling_residue.index)

cad_rolling_residue = rolling_residue.iloc[:,0]
aud_rolling_residue = rolling_residue.iloc[:,1]
gbp_rolling_residue = rolling_residue.iloc[:,2]

rolling_residue

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


### LSTM

In [47]:
model_type = LSTM

#### cad

In [48]:
cad_rolling_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000002
2020-12-28   -0.000002
2020-12-29   -0.000002
2020-12-30   -0.000002
2020-12-31   -0.000001
Name: cad_rolling_linear_prediction, Length: 10435, dtype: float64

In [49]:
cad_rolling_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 10435, dtype: float64

In [50]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [51]:
cad_rolling_LSTM_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [52]:
aud_rolling_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000015
2020-12-28   -0.000014
2020-12-29   -0.000014
2020-12-30   -0.000014
2020-12-31   -0.000013
Name: aud_rolling_linear_prediction, Length: 10435, dtype: float64

In [53]:
aud_rolling_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 10435, dtype: float64

In [54]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 256
num_epochs = 25

In [55]:
aud_rolling_LSTM_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [56]:
gbp_rolling_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000171
2020-12-28    0.000020
2020-12-29   -0.000245
2020-12-30    0.000138
2020-12-31    0.000309
Name: gbp_rolling_linear_prediction, Length: 10435, dtype: float64

In [57]:
gbp_rolling_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 10435, dtype: float64

In [58]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 25
batch_sizes = 256
num_epochs = 5

In [59]:
gbp_rolling_LSTM_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [60]:
model_type = Self_Attention

#### cad

In [61]:
cad_rolling_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000002
2020-12-28   -0.000002
2020-12-29   -0.000002
2020-12-30   -0.000002
2020-12-31   -0.000001
Name: cad_rolling_linear_prediction, Length: 10435, dtype: float64

In [62]:
cad_rolling_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 10435, dtype: float64

In [63]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 15

In [64]:
cad_rolling_Self_Attention_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [65]:
aud_rolling_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000015
2020-12-28   -0.000014
2020-12-29   -0.000014
2020-12-30   -0.000014
2020-12-31   -0.000013
Name: aud_rolling_linear_prediction, Length: 10435, dtype: float64

In [66]:
aud_rolling_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 10435, dtype: float64

In [67]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 1
num_hidden_sizes = 800
batch_sizes = 16
num_epochs = 30

In [68]:
aud_rolling_Self_Attention_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [69]:
gbp_rolling_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000171
2020-12-28    0.000020
2020-12-29   -0.000245
2020-12-30    0.000138
2020-12-31    0.000309
Name: gbp_rolling_linear_prediction, Length: 10435, dtype: float64

In [70]:
gbp_rolling_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 10435, dtype: float64

In [71]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 20

In [72]:
gbp_rolling_Self_Attention_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

# 輸出模型預測與比較

## 輸出模型預測

In [73]:
LSTM_recursive_prediction = pd.DataFrame(np.column_stack((cad_recursive_LSTM_pred, aud_recursive_LSTM_pred, gbp_recursive_LSTM_pred)))
LSTM_recursive_prediction.columns = ['cad_recursive_LSTM_pred', 'aud_recursive_LSTM_pred', 'gbp_recursive_LSTM_pred']
LSTM_recursive_prediction.to_csv('./LSTM_recursive_prediction.csv')
LSTM_recursive_prediction

,cad_recursive_LSTM_pred,aud_recursive_LSTM_pred,gbp_recursive_LSTM_pred
0,-0.000065,0.000897,-0.000205
1,0.000016,0.000791,-0.000549
2,-0.000044,0.000818,-0.000460
3,-0.000117,0.000821,-0.000176
4,-0.000008,0.000760,-0.000203
...,...,...,...
1039,0.000027,0.000976,-0.000232
1040,-0.000058,0.000683,-0.000402
1041,-0.000027,0.000869,-0.000611
1042,-0.000045,0.000745,-0.000250


In [74]:
LSTM_rolling_prediction = pd.DataFrame(np.column_stack((cad_rolling_LSTM_pred, aud_rolling_LSTM_pred, gbp_rolling_LSTM_pred)))
LSTM_rolling_prediction.columns = ['cad_rolling_LSTM_pred', 'aud_rolling_LSTM_pred', 'gbp_rolling_LSTM_pred']
LSTM_rolling_prediction.to_csv('./LSTM_rolling_prediction.csv')
LSTM_rolling_prediction

,cad_rolling_LSTM_pred,aud_rolling_LSTM_pred,gbp_rolling_LSTM_pred
0,-0.000065,0.001197,-0.000205
1,0.000016,0.001142,-0.000549
2,-0.000045,0.001312,-0.000462
3,-0.000117,0.001169,-0.000177
4,-0.000008,0.001096,-0.000204
...,...,...,...
1039,0.000036,0.001275,-0.000168
1040,-0.000048,0.001076,-0.000332
1041,-0.000017,0.001384,-0.000531
1042,-0.000035,0.001236,-0.000184


In [75]:
Self_Attention_recursive_prediction = pd.DataFrame(np.column_stack((cad_recursive_Self_Attention_pred, aud_recursive_Self_Attention_pred, gbp_recursive_Self_Attention_pred)))
Self_Attention_recursive_prediction.columns = ['cad_recursive_Self_Attention_pred', 'aud_recursive_Self_Attention_pred', 'gbp_recursive_Self_Attention_pred']
Self_Attention_recursive_prediction.to_csv('./Self_Attention_recursive_prediction.csv')
Self_Attention_recursive_prediction

,cad_recursive_Self_Attention_pred,aud_recursive_Self_Attention_pred,gbp_recursive_Self_Attention_pred
0,0.000123,-0.001629,-0.000229
1,0.000154,-0.001620,-0.000563
2,0.000076,-0.001619,-0.000485
3,-0.000028,-0.001618,-0.000241
4,-0.000047,-0.001618,-0.000258
...,...,...,...
1039,0.000366,-0.001620,-0.000203
1040,0.000302,-0.001620,-0.000405
1041,0.000212,-0.001621,-0.000644
1042,0.000135,-0.001620,-0.000358


In [76]:
Self_Attention_rolling_prediction = pd.DataFrame(np.column_stack((cad_rolling_Self_Attention_pred, aud_rolling_Self_Attention_pred, gbp_rolling_Self_Attention_pred)))
Self_Attention_rolling_prediction.columns = ['cad_rolling_Self_Attention_pred', 'aud_rolling_Self_Attention_pred', 'gbp_rolling_Self_Attention_pred']
Self_Attention_rolling_prediction.to_csv('./Self_Attention_rolling_prediction.csv')
Self_Attention_rolling_prediction

,cad_rolling_Self_Attention_pred,aud_rolling_Self_Attention_pred,gbp_rolling_Self_Attention_pred
0,0.000123,-0.001629,-0.000229
1,0.000154,-0.001620,-0.000563
2,0.000076,-0.001620,-0.000486
3,-0.000029,-0.001619,-0.000242
4,-0.000048,-0.001618,-0.000259
...,...,...,...
1039,0.000376,-0.001588,-0.000137
1040,0.000313,-0.001588,-0.000334
1041,0.000223,-0.001588,-0.000563
1042,0.000146,-0.001587,-0.000290


## 載入實際匯率變動率（取對數後一階差分）

In [77]:
groundtruth = pd.read_excel('./CAD_AUD_GBP.xlsx', index_col=0, skiprows=3)
groundtruth = groundtruth.iloc[1:,:]
groundtruth = groundtruth.astype('float64')
groundtruth.index = pd.to_datetime(groundtruth.index)
groundtruth = np.log(groundtruth)
groundtruth = groundtruth.diff().dropna()

groundtruth = groundtruth.loc['2017-01-01':'2020-12-31']
cad_groundtruth = groundtruth.iloc[:,0]
aud_groundtruth = groundtruth.iloc[:,1]
gbp_groundtruth = groundtruth.iloc[:,2]

groundtruth

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
2017-01-02,-0.000269,-0.003335,-0.005037
2017-01-03,0.001209,0.004860,-0.003345
2017-01-04,0.009488,0.008826,0.007003
2017-01-05,0.005703,0.007523,0.007760
2017-01-06,-0.000926,-0.005055,-0.010686
...,...,...,...
2020-12-25,-0.000129,0.001052,0.000148
2020-12-28,0.001414,-0.003425,-0.007996
2020-12-29,0.002182,0.003556,0.003784
2020-12-30,0.005242,0.010463,0.008994


## 模型比較（計算模型的 RMSE, MAE, DA）

### Random walk pred

In [78]:
random_walk_pred = rw_pred(cad_groundtruth)

cad_random_walk_RMSE = root_mean_squared_error(random_walk_pred, cad_groundtruth)
cad_random_walk_MAE = mean_absolute_error(random_walk_pred, cad_groundtruth)
cad_random_walk_DA = 0.5
aud_random_walk_RMSE = root_mean_squared_error(random_walk_pred, aud_groundtruth)
aud_random_walk_MAE = mean_absolute_error(random_walk_pred, aud_groundtruth)
aud_random_walk_DA = 0.5
gbp_random_walk_RMSE = root_mean_squared_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_MAE = mean_absolute_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_DA = 0.5

In [79]:
cad_random_walk_RMSE, cad_random_walk_MAE, cad_random_walk_DA

(0.004365277893580916, 0.0032165540648211085, 0.5)

In [80]:
aud_random_walk_RMSE, aud_random_walk_MAE, aud_random_walk_DA

(0.005655793284191722, 0.00416259953667118, 0.5)

In [81]:
gbp_random_walk_RMSE, gbp_random_walk_MAE, gbp_random_walk_DA

(0.0056882604301849125, 0.004202877474568311, 0.5)

### Recursive LSTM pred

In [82]:
cad_recursive_LSTM_RMSE = root_mean_squared_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_MAE = mean_absolute_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_DA = direction_accuracy(cad_recursive_LSTM_pred, cad_groundtruth)
aud_recursive_LSTM_RMSE = root_mean_squared_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_MAE = mean_absolute_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_DA = direction_accuracy(aud_recursive_LSTM_pred, aud_groundtruth)
gbp_recursive_LSTM_RMSE = root_mean_squared_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_MAE = mean_absolute_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_DA = direction_accuracy(gbp_recursive_LSTM_pred, gbp_groundtruth)

In [83]:
cad_recursive_LSTM_RMSE, cad_recursive_LSTM_MAE, cad_recursive_LSTM_DA

(0.004366953448165288, 0.0032166379683476027, 0.49616858237547895)

In [84]:
aud_recursive_LSTM_RMSE, aud_recursive_LSTM_MAE, aud_recursive_LSTM_DA

(0.005716025064109846, 0.00420648458967849, 0.5067049808429118)

In [85]:
gbp_recursive_LSTM_RMSE, gbp_recursive_LSTM_MAE, gbp_recursive_LSTM_DA

(0.005707144618527106, 0.004224464636170401, 0.4827586206896552)

### Rolling LSTM pred

In [86]:
cad_rolling_LSTM_RMSE = root_mean_squared_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_MAE = mean_absolute_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_DA = direction_accuracy(cad_rolling_LSTM_pred, cad_groundtruth)
aud_rolling_LSTM_RMSE = root_mean_squared_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_MAE = mean_absolute_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_DA = direction_accuracy(aud_rolling_LSTM_pred, aud_groundtruth)
gbp_rolling_LSTM_RMSE = root_mean_squared_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_MAE = mean_absolute_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_DA = direction_accuracy(gbp_rolling_LSTM_pred, gbp_groundtruth)

In [87]:
cad_rolling_LSTM_RMSE, cad_rolling_LSTM_MAE, cad_rolling_LSTM_DA

(0.004366874890696316, 0.003216533494752228, 0.4942528735632184)

In [88]:
aud_rolling_LSTM_RMSE, aud_rolling_LSTM_MAE, aud_rolling_LSTM_DA

(0.00577983214485462, 0.0042638320321405545, 0.5067049808429118)

In [89]:
gbp_rolling_LSTM_RMSE, gbp_rolling_LSTM_MAE, gbp_rolling_LSTM_DA

(0.005705038932962677, 0.004221539465330905, 0.4827586206896552)

### Recursive Self-Attention pred

In [90]:
cad_recursive_Self_Attention_RMSE = root_mean_squared_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_MAE = mean_absolute_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_DA = direction_accuracy(cad_recursive_Self_Attention_pred, cad_groundtruth)
aud_recursive_Self_Attention_RMSE = root_mean_squared_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_MAE = mean_absolute_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_DA = direction_accuracy(aud_recursive_Self_Attention_pred, aud_groundtruth)
gbp_recursive_Self_Attention_RMSE = root_mean_squared_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_MAE = mean_absolute_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_DA = direction_accuracy(gbp_recursive_Self_Attention_pred, gbp_groundtruth)

In [91]:
cad_recursive_Self_Attention_RMSE, cad_recursive_Self_Attention_MAE, cad_recursive_Self_Attention_DA

(0.004372101752788658, 0.0032285911199098063, 0.4827586206896552)

In [92]:
aud_recursive_Self_Attention_RMSE, aud_recursive_Self_Attention_MAE, aud_recursive_Self_Attention_DA

(0.005902485725115945, 0.004439521080075167, 0.4779693486590038)

In [93]:
gbp_recursive_Self_Attention_RMSE, gbp_recursive_Self_Attention_MAE, gbp_recursive_Self_Attention_DA

(0.005711377556770118, 0.004226133230130188, 0.4827586206896552)

### Rolling Self-Attention pred

In [94]:
cad_rolling_Self_Attention_RMSE = root_mean_squared_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_MAE = mean_absolute_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_DA = direction_accuracy(cad_rolling_Self_Attention_pred, cad_groundtruth)
aud_rolling_Self_Attention_RMSE = root_mean_squared_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_MAE = mean_absolute_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_DA = direction_accuracy(aud_rolling_Self_Attention_pred, aud_groundtruth)
gbp_rolling_Self_Attention_RMSE = root_mean_squared_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_MAE = mean_absolute_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_DA = direction_accuracy(gbp_rolling_Self_Attention_pred, gbp_groundtruth)

In [95]:
cad_rolling_Self_Attention_RMSE, cad_rolling_Self_Attention_MAE, cad_rolling_Self_Attention_DA

(0.004372219300351518, 0.003228707155933871, 0.4827586206896552)

In [96]:
aud_rolling_Self_Attention_RMSE, aud_rolling_Self_Attention_MAE, aud_rolling_Self_Attention_DA

(0.005897562861361657, 0.004434635820803504, 0.4779693486590038)

In [97]:
gbp_rolling_Self_Attention_RMSE, gbp_rolling_Self_Attention_MAE, gbp_rolling_Self_Attention_DA

(0.005709054168491827, 0.004223106621892312, 0.48563218390804597)

# 殘差檢定

## 線性部分

### recursive

In [101]:
acorr_ljungbox(cad_recursive_residue)

(array([  6.6158301 ,   7.16321564,   7.27857123,  11.48477986,
         25.53262771,  25.59008732,  25.76711743,  28.98012071,
         47.01567674,  48.35475605,  48.53997185,  53.55354627,
         60.0059039 ,  64.02690294,  64.03931675,  70.45652474,
         83.82027845,  86.271629  ,  86.51988296,  87.02649903,
         89.74099268,  99.3569405 , 101.33175007, 103.30067974,
        103.30418138, 108.0045672 , 119.89240611, 119.93909667,
        124.74502602, 128.72997397, 139.57243575, 139.90669345,
        151.28601444, 153.76168737, 154.66434571, 156.29250449,
        156.32561147, 163.24944213, 163.66824001, 176.25852764]),
 array([1.01076215e-02, 2.78309151e-02, 6.35293553e-02, 2.16234808e-02,
        1.09923507e-04, 2.65384436e-04, 5.54266008e-04, 3.19655297e-04,
        3.89835799e-07, 5.34620345e-07, 1.14351099e-06, 3.28283486e-07,
        5.24072432e-08, 2.28040374e-08, 5.03162164e-08, 8.29983321e-09,
        7.97849908e-11, 6.69877853e-11, 1.36370521e-10, 2.44256269e-10

In [102]:
acorr_ljungbox(aud_recursive_residue)

(array([ 4.01244066,  4.15625187,  6.86475738,  9.2037341 , 10.4197658 ,
        10.79814028, 11.26839167, 11.47216529, 12.14964413, 12.33562197,
        14.42498889, 15.1103757 , 17.97286374, 19.2759486 , 19.64260024,
        30.71744113, 38.31795709, 38.37346623, 39.70417439, 39.70453155,
        40.02012647, 40.42296995, 40.49449434, 40.73314327, 41.82459498,
        42.76151355, 43.9281836 , 44.18773927, 49.60116358, 49.7140684 ,
        50.87547816, 51.34263854, 53.9102209 , 54.2664781 , 54.27961894,
        54.46887134, 57.05203641, 59.35161107, 64.02296978, 71.18477359]),
 array([0.04516572, 0.12516456, 0.07633565, 0.05620401, 0.06417842,
        0.09481912, 0.12733282, 0.17635338, 0.20500397, 0.26322231,
        0.2103624 , 0.23545575, 0.15855253, 0.15467077, 0.1860758 ,
        0.01461849, 0.00222202, 0.0034557 , 0.00357768, 0.00544295,
        0.00739498, 0.00964467, 0.01351911, 0.01781523, 0.01876852,
        0.02046077, 0.02103374, 0.02661824, 0.00996736, 0.01327769,
      

In [103]:
acorr_ljungbox(gbp_recursive_residue)

(array([7.43312254e-04, 2.16466347e-01, 5.89851789e-01, 6.24555528e-01,
        8.93862641e-01, 1.82525190e+00, 2.24089096e+00, 2.25874071e+00,
        2.29712631e+00, 4.29493404e+00, 7.50198067e+00, 8.32785043e+00,
        8.38520505e+00, 9.09922492e+00, 1.70208423e+01, 1.71086193e+01,
        1.95303799e+01, 1.95554013e+01, 2.11096033e+01, 2.32214088e+01,
        2.33102478e+01, 2.36185844e+01, 2.37528638e+01, 2.38015855e+01,
        2.38718104e+01, 2.41380085e+01, 2.41851663e+01, 2.47821619e+01,
        2.50102721e+01, 2.50125101e+01, 2.51809184e+01, 2.58720641e+01,
        2.61089055e+01, 2.63858177e+01, 2.86065197e+01, 2.87036541e+01,
        2.93714357e+01, 4.06865271e+01, 4.23715595e+01, 4.24219408e+01]),
 array([0.97824937, 0.89741832, 0.89875159, 0.96029631, 0.97066437,
        0.93504808, 0.94531395, 0.97203426, 0.98585132, 0.93306271,
        0.75709824, 0.75901392, 0.81763125, 0.82464215, 0.31761998,
        0.3786023 , 0.29895106, 0.35839791, 0.33077647, 0.27805413,
      

### rolling

In [104]:
acorr_ljungbox(cad_rolling_residue)

(array([  6.61747327,   7.16444296,   7.27960065,  11.48433713,
         25.53540785,  25.59307784,  25.77045343,  28.98189925,
         47.02143506,  48.35918785,  48.544892  ,  53.56108359,
         60.01075383,  64.03388227,  64.0464188 ,  70.46635905,
         83.8261872 ,  86.27933788,  86.52815941,  87.03397008,
         89.74660334,  99.36594781, 101.34214166, 103.31249038,
        103.31593712, 108.01854369, 119.90310006, 119.94959562,
        124.75355905, 128.7366845 , 139.57635971, 139.91014465,
        151.29235671, 153.76697832, 154.66902624, 156.29801143,
        156.33123729, 163.25348575, 163.67266148, 176.26098969]),
 array([1.00983000e-02, 2.78138417e-02, 6.35002544e-02, 2.16275579e-02,
        1.09787396e-04, 2.65045061e-04, 5.53507152e-04, 3.19425688e-04,
        3.88874167e-07, 5.33623857e-07, 1.14119813e-06, 3.27269279e-07,
        5.23026890e-08, 2.27389398e-08, 5.01727158e-08, 8.26693090e-09,
        7.95905462e-11, 6.67765254e-11, 1.35914845e-10, 2.43528531e-10

In [105]:
acorr_ljungbox(aud_rolling_residue)

(array([ 4.01596825,  4.16038732,  6.87135985,  9.2081671 , 10.42580899,
        10.80503193, 11.27442569, 11.47756438, 12.15626467, 12.34287355,
        14.43028543, 15.11450104, 17.97474771, 19.27933801, 19.64520193,
        30.72419905, 38.32162907, 38.37688449, 39.70640464, 39.70674297,
        40.0218136 , 40.42525915, 40.49700988, 40.73609903, 41.82666898,
        42.76265564, 43.92834674, 44.18751238, 49.59897979, 49.71215878,
        50.8726929 , 51.33927195, 53.90845473, 54.26415597, 54.2771955 ,
        54.46684984, 57.0514814 , 59.34983087, 64.01945411, 71.1790807 ]),
 array([0.04507134, 0.12490602, 0.07611298, 0.05610176, 0.06403093,
        0.09459232, 0.12708858, 0.1760795 , 0.20464255, 0.26276439,
        0.21009142, 0.23523437, 0.15848063, 0.1545478 , 0.18596969,
        0.01458963, 0.00221943, 0.00345208, 0.00357528, 0.00543946,
        0.00739149, 0.00963869, 0.01351025, 0.01780201, 0.01875892,
        0.02045513, 0.02103292, 0.02661962, 0.00997272, 0.01328372,
      

In [106]:
acorr_ljungbox(gbp_rolling_residue)

(array([9.48583839e-04, 2.16356901e-01, 5.91879971e-01, 6.27314416e-01,
        8.93764852e-01, 1.83178910e+00, 2.25172150e+00, 2.27048981e+00,
        2.30747078e+00, 4.31318463e+00, 7.52957928e+00, 8.35957467e+00,
        8.41784062e+00, 9.12871197e+00, 1.70393884e+01, 1.71284292e+01,
        1.95430724e+01, 1.95672571e+01, 2.11281462e+01, 2.32330562e+01,
        2.33208703e+01, 2.36269040e+01, 2.37623272e+01, 2.38121016e+01,
        2.38815133e+01, 2.41503105e+01, 2.41967134e+01, 2.47905251e+01,
        2.50163246e+01, 2.50183760e+01, 2.51883921e+01, 2.58758435e+01,
        2.61142945e+01, 2.63891636e+01, 2.86044067e+01, 2.87025852e+01,
        2.93722357e+01, 4.06783002e+01, 4.23611255e+01, 4.24120181e+01]),
 array([0.97542977, 0.89746743, 0.89828873, 0.9599806 , 0.9706714 ,
        0.93450054, 0.94460541, 0.97157616, 0.98562114, 0.9321147 ,
        0.75472021, 0.75643753, 0.81535417, 0.82271791, 0.31651522,
        0.37733395, 0.29826456, 0.35770186, 0.32976355, 0.27749642,
      

## 線性加上非線性部分

### recursive_LSTM

In [107]:
cad_combined_recursive_LSTM_residue = cad_groundtruth - cad_recursive_LSTM_pred
aud_combined_recursive_LSTM_residue = aud_groundtruth - aud_recursive_LSTM_pred
gbp_combined_recursive_LSTM_residue = gbp_groundtruth - gbp_recursive_LSTM_pred


In [108]:
acorr_ljungbox(cad_combined_recursive_LSTM_residue)

(array([ 0.11039674,  0.7585303 ,  1.87249662,  2.30152472,  2.40810206,
         3.98321142,  4.14740025,  4.33428591,  4.33428821,  5.02019104,
         5.93639632,  6.04428498,  6.99096028,  7.14382444,  7.22808357,
         7.34972535, 11.88175344, 12.42298959, 12.6359387 , 12.65618267,
        14.11160054, 14.24957263, 14.62860817, 14.96615667, 15.02148999,
        20.38794997, 23.90668186, 24.14537847, 24.3932559 , 26.10986757,
        27.99921434, 28.47546256, 33.68386966, 33.71654877, 34.34185166,
        34.35965796, 37.40041248, 37.56853241, 38.75608331, 40.81056588]),
 array([0.73969291, 0.68436413, 0.59928732, 0.68049147, 0.79026696,
        0.67894849, 0.76265193, 0.82577422, 0.88806618, 0.88982516,
        0.87757416, 0.91383322, 0.90261681, 0.92902513, 0.95100867,
        0.96591679, 0.80724401, 0.82464825, 0.85667582, 0.89164369,
        0.86475799, 0.89238517, 0.90744859, 0.92174505, 0.94087881,
        0.77282428, 0.63548738, 0.67383912, 0.70931266, 0.66951188,
      

In [109]:
acorr_ljungbox(aud_combined_recursive_LSTM_residue)

(array([ 8.99483678, 16.46393391, 18.79333934, 19.46077852, 20.83262225,
        23.62422717, 30.91749999, 35.41723078, 36.52383601, 36.54077652,
        38.35238034, 38.62086844, 38.6208689 , 39.40884988, 41.82699272,
        43.22029834, 43.27830493, 43.55222981, 44.89245127, 45.37261299,
        45.50559833, 45.68830255, 45.69796515, 49.46138517, 50.07998174,
        50.12919007, 51.07959248, 52.05687243, 52.20665046, 52.57968769,
        53.128382  , 54.38117211, 54.50044074, 57.60081969, 57.96481704,
        58.60477728, 61.01796861, 66.16589487, 72.6770038 , 72.73214256]),
 array([2.70743455e-03, 2.66012580e-04, 3.01658944e-04, 6.37912503e-04,
        8.71208171e-04, 6.12228443e-04, 6.43902793e-05, 2.24418044e-05,
        3.19974377e-05, 6.79341547e-05, 6.82127656e-05, 1.21487171e-04,
        2.29786208e-04, 3.15271408e-04, 2.38948854e-04, 2.58879525e-04,
        4.37200152e-04, 6.68238809e-04, 7.09560693e-04, 9.82021048e-04,
        1.48586218e-03, 2.16836479e-03, 3.26079275e-03

In [110]:
acorr_ljungbox(gbp_combined_recursive_LSTM_residue)

(array([ 0.17633618,  2.33659131,  3.90246216,  4.57825321,  4.89672036,
         6.49728331, 18.60025401, 19.72924833, 25.3544204 , 27.23356296,
        27.23365748, 28.49508025, 29.35416331, 29.36486226, 29.37616998,
        30.09259296, 32.26130988, 32.2950258 , 32.46910816, 32.97057423,
        33.01842638, 33.42383213, 33.97816585, 35.30410935, 37.33912889,
        37.45405965, 37.68845432, 39.6829887 , 39.93224281, 41.59421624,
        42.25469775, 43.20441615, 45.20059062, 45.29851675, 45.94286445,
        45.96140502, 47.24872004, 50.37113047, 53.08949411, 53.32223389]),
 array([0.67454096, 0.31089636, 0.2721909 , 0.33336925, 0.42861429,
        0.3698449 , 0.00953624, 0.0114104 , 0.00260315, 0.00239163,
        0.00423598, 0.00467939, 0.00582896, 0.00932414, 0.01438079,
        0.01752797, 0.01396129, 0.02028277, 0.02765592, 0.03399245,
        0.0460132 , 0.05615558, 0.06547456, 0.0640171 , 0.05359485,
        0.06799263, 0.08291704, 0.07054647, 0.08505084, 0.07749499,
      

### recursive_Self_Attention

In [111]:
cad_combined_recursive_Self_Attention_residue = cad_groundtruth - cad_recursive_Self_Attention_pred
aud_combined_recursive_Self_Attention_residue = aud_groundtruth - aud_recursive_Self_Attention_pred
gbp_combined_recursive_Self_Attention_residue = gbp_groundtruth - gbp_recursive_Self_Attention_pred


In [112]:
acorr_ljungbox(cad_combined_recursive_Self_Attention_residue)

(array([ 0.18484302,  0.18716811,  2.15895149,  3.44385214,  3.45513934,
         4.9962272 ,  5.16539153,  5.35690052,  5.35772133,  6.00238121,
         6.97530686,  7.11710187,  8.03736006,  8.1980319 ,  8.2881472 ,
         8.44274105, 12.84514818, 13.42465302, 13.61258819, 13.62850228,
        15.13845799, 15.27674508, 15.71544151, 15.98258103, 16.02754566,
        21.48488671, 25.02726786, 25.34993411, 25.56936749, 27.3408097 ,
        29.074122  , 29.56200148, 34.52546185, 34.57270578, 35.26253135,
        35.26812428, 38.41976785, 38.58671253, 39.83942312, 41.85707551]),
 array([0.66724384, 0.91066147, 0.54007884, 0.48646694, 0.63018622,
        0.5442971 , 0.63978765, 0.71883736, 0.80207683, 0.81506315,
        0.80107966, 0.84977887, 0.84115784, 0.87875581, 0.91171149,
        0.93458035, 0.74648911, 0.76575372, 0.80578901, 0.84882129,
        0.81594079, 0.85009017, 0.86717691, 0.88870371, 0.91400602,
        0.71652295, 0.57293379, 0.60872967, 0.64839706, 0.60532456,
      

In [113]:
acorr_ljungbox(aud_combined_recursive_Self_Attention_residue)

(array([ 6.47758259, 15.13518002, 16.36167023, 16.65903023, 17.80683862,
        20.08907579, 27.04831918, 31.65084268, 32.6888996 , 32.69356033,
        34.49901322, 34.68739333, 34.68751374, 35.39678621, 37.83768523,
        39.18640891, 39.23124559, 39.50111151, 40.7449972 , 41.18213361,
        41.2989702 , 41.45907555, 41.47244442, 45.15263381, 45.76778911,
        45.8130421 , 46.75792754, 47.74576942, 47.88556645, 48.29125118,
        48.90306172, 50.11140708, 50.19961605, 53.31835345, 53.7098168 ,
        54.29309065, 56.77290846, 61.55132848, 67.96187373, 68.00357116]),
 array([0.01092435, 0.00051694, 0.0009559 , 0.00225111, 0.00319849,
        0.00267008, 0.00032667, 0.00010755, 0.00015127, 0.00030663,
        0.00029961, 0.0005251 , 0.00094544, 0.00128372, 0.00095348,
        0.00102222, 0.00165928, 0.00243932, 0.00260934, 0.00352967,
        0.00514837, 0.00726333, 0.01045143, 0.00558771, 0.00682744,
        0.00957071, 0.01052997, 0.01141158, 0.01510456, 0.01853835,
      

In [114]:
acorr_ljungbox(gbp_combined_recursive_Self_Attention_residue)

(array([ 0.24176547,  2.56442612,  4.39959629,  5.32919805,  5.47020951,
         7.10198974, 19.12559351, 20.26916421, 25.97250521, 27.94024713,
        27.94197568, 29.10068443, 30.00694299, 30.02419992, 30.03692439,
        30.75591407, 32.89039339, 32.92537046, 33.08650283, 33.58599371,
        33.63416658, 34.02540438, 34.56710619, 35.86880629, 37.91761353,
        38.02517495, 38.25542815, 40.26142997, 40.52286975, 42.2223666 ,
        42.89664834, 43.82455984, 45.81548996, 45.90605917, 46.55119452,
        46.56473064, 47.89323862, 51.04754264, 53.79082292, 54.01423035]),
 array([0.6229339 , 0.27742267, 0.22142282, 0.25515601, 0.36122358,
        0.31151821, 0.00780288, 0.0093643 , 0.00206426, 0.00184544,
        0.00330425, 0.00380658, 0.00469887, 0.00757355, 0.01178904,
        0.01445492, 0.01163753, 0.01704078, 0.02349366, 0.02906554,
        0.03964131, 0.04883289, 0.05739068, 0.05651659, 0.04712486,
        0.06023625, 0.07391323, 0.06266339, 0.07574415, 0.06849433,
      

### rolling_LSTM

In [115]:
cad_combined_rolling_LSTM_residue = cad_groundtruth - cad_rolling_LSTM_pred
aud_combined_rolling_LSTM_residue = aud_groundtruth - aud_rolling_LSTM_pred
gbp_combined_rolling_LSTM_residue = gbp_groundtruth - gbp_rolling_LSTM_pred


In [116]:
acorr_ljungbox(cad_combined_rolling_LSTM_residue)

(array([ 0.11012607,  0.75882426,  1.87187575,  2.29996405,  2.40713516,
         3.98467343,  4.14814898,  4.33588367,  4.33588391,  5.01948866,
         5.93829731,  6.04525282,  6.99430873,  7.14626741,  7.22982008,
         7.35232333, 11.8788234 , 12.4221705 , 12.63642672, 12.6570759 ,
        14.11602903, 14.25296267, 14.63356117, 14.96958753, 15.02435456,
        20.39698471, 23.91123383, 24.15105685, 24.40009044, 26.11987668,
        28.0061829 , 28.48096847, 33.69438751, 33.72677026, 34.35086551,
        34.36886864, 37.40711928, 37.57470731, 38.7608873 , 40.81364695]),
 array([0.74000065, 0.68426355, 0.59942023, 0.6807756 , 0.79041108,
        0.67875063, 0.76256423, 0.82561903, 0.88794847, 0.88987236,
        0.87744933, 0.91378371, 0.90244462, 0.92892608, 0.9509557 ,
        0.96585761, 0.80741873, 0.82469341, 0.85665228, 0.89160793,
        0.86456392, 0.8922584 , 0.90728423, 0.92164541, 0.94081144,
        0.77238   , 0.63523537, 0.67353761, 0.70896783, 0.66899878,
      

In [117]:
acorr_ljungbox(aud_combined_rolling_LSTM_residue)

(array([ 7.01007467, 15.80880998, 18.01801931, 18.92378784, 19.73679649,
        21.99710679, 29.02062664, 33.50836377, 34.56785811, 34.57322111,
        36.33181076, 36.5803416 , 36.5807208 , 37.29849534, 39.72251268,
        41.0741644 , 41.12438687, 41.39752324, 42.70788962, 43.17171056,
        43.30504838, 43.4854977 , 43.50136117, 47.26320106, 47.91038083,
        47.94933949, 48.89731255, 49.97926301, 50.10090055, 50.45887145,
        50.9905521 , 52.24654598, 52.34250141, 55.38470427, 55.74860027,
        56.36499574, 58.76066125, 63.71928447, 70.16885036, 70.21093402]),
 array([8.10522997e-03, 3.69114011e-04, 4.36101756e-04, 8.13508311e-04,
        1.40010639e-03, 1.21233590e-03, 1.43439167e-04, 4.98619827e-05,
        7.09981014e-05, 1.47624151e-04, 1.49021280e-04, 2.61226181e-04,
        4.81847559e-04, 6.64730182e-04, 4.99350040e-04, 5.41454156e-04,
        8.96024159e-04, 1.34137466e-03, 1.42091835e-03, 1.94013440e-03,
        2.87337566e-03, 4.11032312e-03, 6.03600416e-03

In [118]:
acorr_ljungbox(gbp_combined_rolling_LSTM_residue)

(array([ 0.19580725,  2.3782424 ,  3.94865174,  4.6260276 ,  4.94617939,
         6.55616356, 18.68105455, 19.81857694, 25.46360102, 27.34614494,
        27.34625248, 28.60590337, 29.4624115 , 29.47294011, 29.48413292,
        30.19956602, 32.37013842, 32.40282873, 32.57893839, 33.07833301,
        33.12667304, 33.53145045, 34.08735649, 35.40905721, 37.43774773,
        37.55142278, 37.78471391, 39.77944067, 40.03030479, 41.69200959,
        42.35238805, 43.29952883, 45.29172232, 45.38972737, 46.034242  ,
        46.05246508, 47.34616   , 50.47455923, 53.20377944, 53.4335783 ]),
 array([0.65812661, 0.30448873, 0.26706246, 0.32786429, 0.42248382,
        0.36384727, 0.00924741, 0.01104466, 0.00249895, 0.0022952 ,
        0.00407264, 0.00450653, 0.00562517, 0.00901316, 0.0139238 ,
        0.01699452, 0.01353063, 0.01969087, 0.02686965, 0.03307886,
        0.04483124, 0.0547791 , 0.06390557, 0.06256147, 0.05243993,
        0.06661284, 0.08132649, 0.0691762 , 0.08344263, 0.07603156,
      

### rolling_Self_Attention

In [119]:
cad_combined_rolling_Self_Attention_residue = cad_groundtruth - cad_rolling_Self_Attention_pred
aud_combined_rolling_Self_Attention_residue = aud_groundtruth - aud_rolling_Self_Attention_pred
gbp_combined_rolling_Self_Attention_residue = gbp_groundtruth - gbp_rolling_Self_Attention_pred


In [120]:
acorr_ljungbox(cad_combined_rolling_Self_Attention_residue)

(array([ 0.18444274,  0.18680677,  2.15720114,  3.44033016,  3.4514095 ,
         4.99514874,  5.16350042,  5.35598295,  5.35687177,  5.99907228,
         6.97496486,  7.11557438,  8.03847458,  8.19809348,  8.28738244,
         8.44307334, 12.83938181, 13.42131647, 13.6106199 , 13.62693324,
        15.14087351, 15.27800312, 15.71859   , 15.98421195, 16.02860045,
        21.49289979, 25.03024705, 25.35438664, 25.57503434, 27.35005622,
        29.08012893, 29.56636529, 34.53525107, 34.58209126, 35.27047712,
        35.27619563, 38.42497016, 38.59131575, 39.84245461, 41.85821685]),
 array([0.66758269, 0.91082601, 0.54042753, 0.48700908, 0.63075241,
        0.54443548, 0.64001813, 0.71893825, 0.8021557 , 0.81534118,
        0.80110726, 0.84988229, 0.84108472, 0.87875245, 0.9117449 ,
        0.93456879, 0.74687027, 0.76596099, 0.8058987 , 0.84889648,
        0.81581736, 0.85003372, 0.86704817, 0.88864503, 0.91397441,
        0.71609599, 0.57276682, 0.60848592, 0.64809723, 0.60483644,
      

In [121]:
acorr_ljungbox(aud_combined_rolling_Self_Attention_residue)

(array([ 6.47252231, 15.12491893, 16.34958251, 16.64790264, 17.79748797,
        20.08222227, 27.04536337, 31.65278591, 32.68838699, 32.69288065,
        34.49552438, 34.68266018, 34.68275633, 35.39012151, 37.8275026 ,
        39.17891994, 39.22423327, 39.4931898 , 40.73877666, 41.17677971,
        41.29326869, 41.45392513, 41.46741091, 45.14513219, 45.76144125,
        45.80710598, 46.75058997, 47.73964809, 47.88003997, 48.28670704,
        48.89743286, 50.10394523, 50.1928742 , 53.3159906 , 53.7061328 ,
        54.287784  , 56.7646977 , 61.53938402, 67.94508186, 67.98640761]),
 array([0.01095549, 0.0005196 , 0.00096138, 0.00226232, 0.00321122,
        0.00267759, 0.00032707, 0.00010746, 0.0001513 , 0.00030671,
        0.00030001, 0.00052601, 0.00094702, 0.00128665, 0.00095678,
        0.00102477, 0.00166303, 0.00244534, 0.00261431, 0.00353528,
        0.00515678, 0.00727367, 0.01046541, 0.00559916, 0.00683898,
        0.00958529, 0.01054941, 0.01142883, 0.01512446, 0.01855783,
      

In [122]:
acorr_ljungbox(gbp_combined_rolling_Self_Attention_residue)

(array([ 0.26454719,  2.61060851,  4.45115314,  5.38294324,  5.52500329,
         7.16586686, 19.21096787, 20.36311158, 26.08643535, 28.05774176,
        28.05952763, 29.21642222, 30.12011757, 30.13716752, 30.1497743 ,
        30.86775453, 33.00405128, 33.0379901 , 33.20107694, 33.69849938,
        33.74715169, 34.1377404 , 34.68094342, 35.978397  , 38.02086094,
        38.12721663, 38.35638378, 40.36254491, 40.62562637, 42.32480004,
        42.99897064, 43.9243961 , 45.91126458, 46.00188512, 46.64728443,
        46.66053712, 47.99561334, 51.15611806, 53.91039154, 54.13086376]),
 array([0.60701325, 0.27109004, 0.21668892, 0.25021207, 0.35521197,
        0.30578008, 0.00755153, 0.0090462 , 0.0019775 , 0.00176722,
        0.00317006, 0.00365858, 0.00452554, 0.00730673, 0.01139274,
        0.01398905, 0.0112583 , 0.01651459, 0.02278728, 0.02823804,
        0.03856141, 0.04756258, 0.05593229, 0.05515285, 0.04604508,
        0.05893381, 0.07240003, 0.06136583, 0.07421586, 0.06711406,
      